In [1]:
# run thru all

#  Glassdoor to get companies & scores
#  Twitter to get handles
#  PI for PI
#  NLU for NLU

In [75]:
from bs4 import BeautifulSoup
import json
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import sys
import operator
import twitter
import re
import time

from watson_developer_cloud import PersonalityInsightsV2 as PersonalityInsights 
from watson_developer_cloud import NaturalLanguageUnderstandingV1
import watson_developer_cloud.natural_language_understanding.features.v1 \
  as Features

from urllib.request import urlopen
import urllib.request
    
    
twitter_consumer_key = '*******************'
twitter_consumer_secret = ''*******************''
twitter_access_token = ''*******************'-'*******************''
twitter_access_secret = ''*******************''

twitter_api = twitter.Api(consumer_key=twitter_consumer_key, consumer_secret=twitter_consumer_secret, access_token_key=twitter_access_token, access_token_secret=twitter_access_secret)


In [76]:
## Glassdoor
ID=''*******************''
Key=''*******************''
IP = ''*******************''
user_agent ='Chrome/%260.0.3112.90'

pull = 500     # how many pages/companies you will pull
action = 'employers'
glassdoor_companies = pd.DataFrame()
key_list_emp = ['name','numberOfRatings','overallRating','ratingDescription','cultureAndValuesRating','seniorLeadershipRating','compensationAndBenefitsRating','careerOpportunitiesRating','workLifeBalanceRating','recommendToFriendRating','sectorName']
key_list_ceo = ['pctApprove','pctDisapprove']
key_list = key_list_emp + key_list_ceo
for key in key_list:
    glassdoor_companies[key] = []
    
for p in range(1,pull):
    url = ("http://api.glassdoor.com/api/api.htm?t.p="+ID+"&t.k="+ Key+"&userip="+IP+"&format=json&v=1&action="+action+"&ps=1&pn=" + str(p))
    headers = {'user-agent': 'Chrome/%260.0.3112.90'}
    try:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        #print(soup)
        newDictionary=json.loads(str(soup))
        if p % 10 == 0:
            print(p)
        try:
            for key in key_list_emp:
                glassdoor_companies.loc[p,key] = newDictionary['response']['employers'][0][key]
        except Exception:
            pass
        try:
            for ceo in key_list_ceo:    
                glassdoor_companies.loc[p,ceo] = newDictionary['response']['employers'][0]['ceo'][ceo]
        except:
            pass
    except:
        pass
    
glassdoor_companies= glassdoor_companies.apply(pd.to_numeric, errors='ignore')

# clean up bad formats in industry column
glassdoor_companies['sectorName'] =glassdoor_companies['sectorName'].str.replace('amp;', '')
glassdoor_companies['name'] =glassdoor_companies['name'].str.replace('amp;', '').str.replace(';','')
glassdoor_companies = glassdoor_companies.query('numberOfRatings>0')
company_list=glassdoor_companies.name.values
company_search=glassdoor_companies.name.values
j=0
i = 0
import re
for i in company_list:
    company_list[j]=i.replace(';','').replace('\'','').replace(' ','').replace('.','').replace('&','').replace('-','')
    company_search[j]=i.replace(';','').replace('\'','').replace(' ','%20')
    j+=1
print(company_list)
# download list:
filename = 'Glassdoor_list.csv'
glassdoor_companies.to_csv(filename, index=False, encoding='utf-8')



10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
['Walmart' 'Target' 'Wells%20Fargo' 'US%20Army' 'AT&T'
 'Bank%20of%20America' 'The%20Home%20Depot' 'IBM' 'Best%20Buy' 'Macys'
 'Lowes' 'Microsoft' 'Amazon' 'McDonalds' 'CVS%20Health' 'Walgreens'
 'J.P.%20Morgan' 'US%20Air%20Force' 'US%20Navy' 'Apple' 'Deloitte'
 'Hewlett-Packard' 'Sears' 'Cisco%20Systems' 'UPS' 'Accenture'
 'Verizon%20Wireless' 'Nordstrom' 'Oracle' 'Enterprise' 'PwC'
 'Intel%20Corporation' 'Kohls' 'Comcast' 'Kroger' 'Sprint' 'Verizon' 'EY'
 'T-Mobile' 'Boeing' 'Cognizant%20Technology%20Solutions'
 'J.%20C.%20Penney' 'Insight%20Global' 'State%20Farm' 'US%20Marine%20Corps'
 'UnitedHealth%20Group' 'Dell' 'Citi' 'Staples' 'Chase'
 'Whole%20Foods%20Market' 'YMCA' 'Google' 'PepsiCo' 'Lockheed%20Martin'
 'US%20Postal%20Service' 'Kaiser%20Permanente' 'Xerox' 'Capital%20One'
 'Panera%20Bread' '

In [77]:
#company_list=glassdoor_companies.name.values

#glassdoor_companies.shape[0]

print(len(glassdoor_companies))
glassdoor_companies.head()

filename = 'glassdoor_ratings.csv'
glassdoor_companies.to_csv(filename, index=False, encoding='utf-8')

498


In [78]:
#print(company_search)

In [79]:
## Get Twitter sites
query_list = []
good_handles_list = []

#company_search = ['walmart','target','att','USNavy', 'Apple', 'Deloitte', 'HewlettPackard' ,'Sears', 'CiscoSystems' ,'UPS']

for query in company_search:
    print(query)
    html = "https://www.google.com/search?site=&source=hp&q="+query+"%20careers&gws_rd=ssl"
    req = urllib.request.Request(html, headers={'User-Agent': 'Chrome/%260.0.3112.90'})
    try: 
        soup = BeautifulSoup(urlopen(req).read(),"html.parser")
    except:
        print("Connection refused by the server..")
        time.sleep(5)
        print("Was a nice sleep, now let me continue...")
        continue
            
    reg=re.compile(".*&sa=")
    links = []
    for item in soup.find_all('h3', attrs={'class' : 'r'}):

        line = (reg.match(item.a['href'][7:]).group())
        links.append(line[:-4])
    career_site = links[0]
    #print(career_site)
    site = career_site
    try:
        page = requests.get(site,allow_redirects=False)
    except:
        print("Connection refused by the server..")
        time.sleep(5)
        print("Was a nice sleep, now let me continue...")
        continue
        
    soup = BeautifulSoup(page.content, 'html.parser')
    for a in soup.find_all('a', href=True):
        tw = a['href']
        if 'twitter.com' in tw:           
            tw = tw.replace('https://twitter.com/@','@') 
            tw = tw.replace('https://www.twitter.com/@','@') 
            tw = tw.replace('http://twitter.com/@','@')
            tw = tw.replace('http://www.twitter.com/@','@')
            tw = tw.replace('http://twitter.com/','@')
            tw = tw.replace('https://twitter.com/','@') 
            tw = tw.replace('http://www.twitter.com/','@')
            tw = tw.replace('https://www.twitter.com/','@') 
            tw = tw.replace('//twitter.com/','@')
            if '?' in tw:
                pass
            else:
                if '/' in tw:
                    pass
                else:
                    if tw in good_handles_list:         #avoid duplicates
                        pass
                    else:
                        good_handles_list.append(tw)
                        query_list.append(query)
        else:
            pass

Walmart
Target
Wells%20Fargo
US%20Army
AT&T
Bank%20of%20America
The%20Home%20Depot
IBM
Best%20Buy
Macys
Lowes
Microsoft
Amazon
McDonalds
CVS%20Health
Walgreens
J.P.%20Morgan
US%20Air%20Force
US%20Navy
Apple
Deloitte
Hewlett-Packard
Sears
Cisco%20Systems
UPS
Accenture
Verizon%20Wireless
Nordstrom
Oracle
Enterprise
PwC
Intel%20Corporation
Kohls
Comcast
Kroger
Sprint
Verizon
EY
T-Mobile
Boeing
Cognizant%20Technology%20Solutions
J.%20C.%20Penney
Insight%20Global
State%20Farm
US%20Marine%20Corps
UnitedHealth%20Group
Dell
Citi
Staples
Chase
Whole%20Foods%20Market
YMCA
Google
PepsiCo
Lockheed%20Martin
US%20Postal%20Service
Kaiser%20Permanente
Xerox
Capital%20One
Panera%20Bread
Infosys
Publix
KPMG
Victorias%20Secret%20Stores
GE
ADP
Dollar%20General
Abercrombie%20&%20Fitch
Tata%20Consultancy%20Services
Marriott%20International
Fidelity%20Investments
U.S.%20Bank
Morgan%20Stanley
Kmart
Old%20Navy
Booz%20Allen%20Hamilton
Northwestern%20Mutual
PNC%20Financial%20Services%20Group
Aramark
Subway
North

C:\Users\boyerm\AppData\Local\Continuum\Anaconda3\lib\site-packages\urllib3\connection.py:344: SubjectAltNameWarning: Certificate for jobs.rutgers.edu has no `subjectAltName`, falling back to check for a `commonName` for now. This feature is being removed by major browsers and deprecated by RFC 2818. (See https://github.com/shazow/urllib3/issues/497 for details.)
  SubjectAltNameWarning


Connection refused by the server..
Was a nice sleep, now let me continue...
Hy-Vee
Texas%20Roadhouse
GE%20Healthcare
Eaton
Aéropostale
Connection refused by the server..
Was a nice sleep, now let me continue...
Darden
Purdue%20University
Yelp
Barclays
LabCorp
CA%20Technologies
JLL
The%20Fresh%20Market
University%20of%20Pennsylvania


C:\Users\boyerm\AppData\Local\Continuum\Anaconda3\lib\site-packages\urllib3\connection.py:344: SubjectAltNameWarning: Certificate for jobs.hr.upenn.edu has no `subjectAltName`, falling back to check for a `commonName` for now. This feature is being removed by major browsers and deprecated by RFC 2818. (See https://github.com/shazow/urllib3/issues/497 for details.)
  SubjectAltNameWarning


Connection refused by the server..
Was a nice sleep, now let me continue...
BB&T
Edelman
Panda%20Express
Rent-A-Center
Sprouts%20Farmers%20Market
Kindred%20Healthcare
CompuCom
MarketSource
General%20Mills
Harvard%20University
New%20York%20City%20Department%20of%20Education
SpaceX
The%20Childrens%20Place
IKEA
Texas%20A&M
Willis%20Towers%20Watson
Hospital%20Corporation%20of%20America
University%20of%20Illinois%20at%20Urbana-Champaign
OReilly%20Auto%20Parts
Toyota%20North%20America
Asurion
Hobby%20Lobby
MIT
Genesis%20HealthCare
ITT%20Technical%20Institute
Frontier%20Communications
Red%20Robin
Penn%20State%20University
Acosta
Republic%20Services
Stryker
hhgregg
Shell
FIS
Regions%20Financial
ExxonMobil
7-Eleven
Emerson
University%20of%20Florida
Charter%20Communications
U.S.%20Cellular
Johns%20Hopkins%20University
Avis%20Budget%20Group
Ameriprise
REI
Avaya
Shoprite%20Supermarkets
Fresenius%20Medical%20Care%20-%20North%20America
Giant%20Eagle
Cargill
Ruby%20Tuesday
McKinsey%20&%20Company
Nort

In [80]:
twitter_career = pd.DataFrame({'name': query_list,'handle':good_handles_list})  
twitter_career['handle']=twitter_career['handle'].str.lower()

twitter_career.head(20)
twitter_career.shape[0]



367

In [81]:
filename = 'twitter_glassdoor.csv'
twitter_career.to_csv(filename, index=False, encoding='utf-8')

In [83]:
## Natural Language Understanding
#The IBM Bluemix credentials for Natural Language Understanding!
nl_username = "'*******************'"
nl_password = "'*******************'"

natural_language_understanding = NaturalLanguageUnderstandingV1(
  username=nl_username,
  password=nl_password,  
    version="2017-02-27")

count = 100   # tweets for NLU
i = 0
text = ""
all_text = []
key_list = ['sadness','joy','fear','disgust','anger']

sadness_list = []
joy_list = []
fear_list = []
disgust_list = []
anger_list = []

tweet_count_list = []
handles_list = []

for handle in good_handles_list:
    b = ''
    all_text = []
    status = ''
    tweets = 0
    print(handle)
    try:
        status = twitter_api.GetUserTimeline(screen_name=handle, count=count, include_rts=False)  # no retweets for analysis

        for x in status:
            clean_text = x.text
            clean_text = re.sub(r"http\S+", "", clean_text)
            clean_text = re.sub(r"@\S+", "", clean_text)
            all_text.append(clean_text)
            tweets +=1

        b = ''.join(all_text)

        nl_result = natural_language_understanding.analyze(text=b,features=[Features.Emotion()  ])

        nl_result['handle'] = handle
        nl_result['all_text'] = b

        i+=1

        sadness = nl_result['emotion']['document']['emotion']['sadness']
        sadness_list.append(sadness)
        joy = nl_result['emotion']['document']['emotion']['joy']
        joy_list.append(joy)
        fear = nl_result['emotion']['document']['emotion']['fear']
        fear_list.append(fear)
        disgust = nl_result['emotion']['document']['emotion']['disgust']
        disgust_list.append(disgust)
        anger = nl_result['emotion']['document']['emotion']['anger']
        anger_list.append(anger)
        tweet_count_list.append(tweets)
        handles_list.append(handle)
    except:
        pass
    
emotion_df = pd.DataFrame({'handle': handles_list ,'tweets':tweet_count_list,'sadness': sadness_list,'joy': joy_list, 'fear': fear_list, 'disgust': disgust_list, 'anger':anger_list})  

emotion_df['handle']=emotion_df['handle'].str.lower()

filename = 'Glassdoor_twitter_natural_language.csv'
emotion_df.to_csv(filename, index=False, encoding='utf-8')

@WalmartCareers
@target
@WellsFargoJobs
@dictionarycom
@HomeDepotCareer
@macysJOBS
@lowescareers
@MicrosoftJobs
@CVSCareers
@cvshealth
@walgreens
@usafrotc
@AmericasNavy
@lifeatdeloitte
@deloitteus
@hpcareers
@HP
@searsholdings
@wearecisco
@UPSjobs
@accentureusjobs
@VerizonCareers
@verizonnews
@VZFoundation
@verizon
@verizonfios
@Vzenterprise
@ExploreOracle
@ehcareersgo
@pwc_us_careers
@JobsatIntel
@intel
@kohlscareers
@kroger
@sprintcareers
@tmobilecareers
@cognizant
@jcpcareers
@insightglobal
@share
@StateFarm
@militarydotcom
@uhgcareers
@citi
@staplescareers
@WFMcareers
@lifeatgoogle
@GreenParkUK
@PepsiCoJOBS
@
@PepsiCo
@WPC
@PepsicoJOBS
@kpcareers
@Xerox
@PublixJobs
@publixjobs
@Publix
@kpmg_us
@ADPCareers
@DGCareers
@usbank
@OldNavy
@GapInc
@lifeatbooz
@BoozAllen
@PNCBank_Help
@pncbank
@Aramark
@ChipotleJobs
@AmericanExpress
@DISHCareers
@Forever21
@JobsWithUs
@thomsonreuters
@_SpectrumJobs
@ChickfilA
@safeway
@allstate
@qualcomm
@pizzahut
@LibertyMutual
@COCACOLACO
@uber
@CSCGlob

In [84]:
emotion_df.head()

,anger,disgust,fear,handle,joy,sadness,tweets
0,0.027637,0.041793,0.052133,@target,0.710294,0.066762,98
1,0.078106,0.078799,0.077643,@wellsfargojobs,0.554264,0.558970,98
2,0.124485,0.105139,0.079033,@dictionarycom,0.610125,0.506170,97
3,0.052529,0.035499,0.050337,@homedepotcareer,0.614395,0.082604,52
4,0.071566,0.054710,0.035488,@macysjobs,0.712694,0.433592,82


In [34]:
## Personality Insights, combine w/NLU?

#watson_username= "39bf0bfe-a62f-480d-bc8b-510d930ed888"
#watson_password= "8ZapWoYe5pma"

def flatten(orig):
    data = {}
    for c in orig['tree']['children']:
        if 'children' in c:
            for c2 in c['children']:
                if 'children' in c2:
                    for c3 in c2['children']:
                        if 'children' in c3:
                            for c4 in c3['children']:
                                if (c4['category'] == 'personality'):
                                    data[c4['id']] = c4['percentage']
                                    if 'children' not in c3:
                                        if (c3['category'] == 'personality'):
                                                data[c3['id']] = c3['percentage']
    return data

i = 0
company_PI = pd.DataFrame()
row = -1
all_text = []
xxtext = []
count = 100

company_PI = pd.DataFrame()
row = -1
for handle in good_handles_list:
    print(handle)
    all_text = []
    status = ''
    b = ''
    
    #get_tweets(handle)
    try:
        status = twitter_api.GetUserTimeline(screen_name=handle, count=count, include_rts=False)
    
        #append_tweets(status)
        for x in status:
            clean_text = x.text
            clean_text = re.sub(r"http\S+", "", clean_text)
            clean_text = re.sub(r"@\S+", "", clean_text)
            all_text.append(clean_text)

        b = ''.join(all_text)

        #personality_insights(b)
        pi_result = PersonalityInsights(username=watson_username, password=watson_password).profile(b)
        f = flatten(pi_result)

        print(f)
        f['handle'] = handle
        f['all_text'] = b

        #print(f)
        #make dataframe for each handle  -- this is NOT working.... maybe put in a function?
        row += 1
        for key in f.keys():
            company_PI.loc[row,key] = f[key]
    except:
        continue


@WalmartCareers
@target
{'Adventurousness': 0.9384073856460708, 'Artistic interests': 0.875146913438752, 'Emotionality': 0.9436435769724698, 'Imagination': 0.03306725204536548, 'Intellect': 0.7605474312052114, 'Liberalism': 0.11477153515859778, 'Achievement striving': 0.9110429019363365, 'Cautiousness': 0.9375545348798504, 'Dutifulness': 0.9646744455644816, 'Orderliness': 0.8940968919619676, 'Self-discipline': 0.9954880264521928, 'Self-efficacy': 0.4550756299188957, 'Activity level': 0.8863594690749363, 'Assertiveness': 0.8485001337890044, 'Cheerfulness': 0.99611400556916, 'Excitement-seeking': 0.13047591498609, 'Friendliness': 0.996945492388918, 'Gregariousness': 0.9973253213136505, 'Altruism': 0.998164256454967, 'Cooperation': 0.9992312715426998, 'Modesty': 0.9788367036439827, 'Morality': 0.9999693991316283, 'Sympathy': 0.9620410649194234, 'Trust': 0.9243197537235974, 'Anger': 0.0037170334320464904, 'Anxiety': 0.0023778567390557726, 'Depression': 0.012185039092510508, 'Immoderation':

In [35]:
## Combine into one df
company_PI['handle']=company_PI['handle'].str.lower()
watson = pd.merge(emotion_df, company_PI, how='outer', on='handle')

print(watson)


        anger   disgust      fear            handle       joy   sadness  \
0    0.027637  0.041793  0.052133           @target  0.710294  0.066762   
1    0.078106  0.078799  0.077643   @wellsfargojobs  0.554264  0.558970   
2    0.124485  0.105139  0.079033    @dictionarycom  0.610125  0.506170   
3    0.052529  0.035499  0.050337  @homedepotcareer  0.614395  0.082604   
4    0.071566  0.054710  0.035488        @macysjobs  0.712694  0.433592   
5    0.060821  0.055594  0.057706     @lowescareers  0.637549  0.115763   
6    0.075412  0.059852  0.075395    @starbucksjobs  0.699406  0.117477   
7    0.042158  0.039420  0.050017        @starbucks  0.739033  0.080786   
8    0.058907  0.044883  0.075683    @microsoftjobs  0.677602  0.082059   
9    0.071444  0.052865  0.051579       @cvscareers  0.632543  0.105776   
10   0.043514  0.031744  0.047498        @cvshealth  0.583910  0.080211   
11   0.044741  0.037915  0.053600        @walgreens  0.671459  0.454694   
12   0.075933  0.060724  

In [ ]:
filename = 'gd_tw_nlu_pi.csv'
watson.to_csv(filename, index=False, encoding='utf-8')

### Construct single dataset

In [93]:
watson_match = pd.merge(watson,twitter_career, how = 'left', on='handle' )
glassdoor_match = pd.merge(glassdoor_companies,twitter_career, how = 'right', left_on='name',right_on='name' )
watson_glassdoor = pd.merge(watson_match,glassdoor_match, how = 'outer', on='handle')
watson_glassdoor = watson_glassdoor.drop_duplicates(keep=False)

watson_glassdoor.tail(20)

,anger,disgust,fear,handle,joy,sadness,tweets,Adventurousness,Artistic interests,Emotionality,...,ratingDescription,cultureAndValuesRating,seniorLeadershipRating,compensationAndBenefitsRating,careerOpportunitiesRating,workLifeBalanceRating,recommendToFriendRating,sectorName,pctApprove,pctDisapprove
494,0.032151,0.021253,0.045514,@wawa,0.650018,0.066659,100.0,0.545233,0.204682,0.829480,...,Satisfied,3.9,3.0,3.7,3.6,3.0,67.0,Retail,82.0,18.0
495,0.073110,0.048440,0.068422,@mercer,0.609864,0.105478,73.0,0.930887,0.123720,0.159044,...,OK,3.4,2.9,3.0,3.2,3.3,62.0,Business Services,80.0,20.0
496,NaN,NaN,NaN,@jobswithus,NaN,NaN,NaN,0.934517,0.088936,0.003045,...,Satisfied,3.7,3.1,3.3,3.2,4.0,72.0,Information Technology,82.0,18.0
497,NaN,NaN,NaN,@pier1,NaN,NaN,NaN,0.606292,0.313120,0.620575,...,OK,3.4,2.9,2.5,2.7,3.2,55.0,Retail,82.0,18.0
498,NaN,NaN,NaN,@teletech,NaN,NaN,NaN,0.983497,0.462531,0.284108,...,OK,3.2,2.9,2.9,3.1,3.2,58.0,Business Services,71.0,29.0
499,NaN,NaN,NaN,@cgi_global,NaN,NaN,NaN,0.544473,0.822527,0.042433,...,OK,3.4,3.0,3.0,3.2,3.6,68.0,Business Services,84.0,16.0
500,NaN,NaN,NaN,@walmartcareers,NaN,NaN,NaN,NaN,NaN,NaN,...,OK,3.0,2.7,3.2,3.3,2.9,56.0,Retail,68.0,32.0
501,NaN,NaN,NaN,@share,NaN,NaN,NaN,NaN,NaN,NaN,...,OK,3.2,2.7,3.3,3.2,3.1,52.0,Insurance,49.0,51.0
502,NaN,NaN,NaN,@cocacolaco,NaN,NaN,NaN,NaN,NaN,NaN,...,Satisfied,3.4,2.9,3.6,3.2,3.2,65.0,Manufacturing,80.0,20.0
503,NaN,NaN,NaN,@kellyjobs,NaN,NaN,NaN,NaN,NaN,NaN,...,OK,3.1,3.0,2.9,3.1,3.3,64.0,Business Services,82.0,18.0


In [95]:
watson_glassdoor.shape

(354, 52)

In [96]:
filename = 'watson_glassdoor.csv'
watson_glassdoor.to_csv(filename, index=False, encoding='utf-8')

In [259]:
shortlist = ('@HomeDepotCareer','@macysJOBS','@lowescareers','@MicrosoftJobs','@CVSCareers','@cvshealth','@walgreens')
handleslist = []
friendsct = []
followerscount = []
verified_list = []
year_started_list = []
status_ct_list = []
fav_ct_list = []

for handle in shortlist: #good_handles_list:            #
    print(handle)
    try:

        status = twitter_api.GetUser(screen_name=handle)

        verified = status.verified
        verified_list.append(verified)

        followers = status.followers_count
        followerscount.append(followers)

        friends = status.friends_count
        friendsct.append(friends)

        year_started = status.created_at[-4:]
        years_on == (2017 - year_started)
        print(years_on)
        year_started_list.append(year_started)
        

        statuses = status.statuses_count
        status_ct_list.append(statuses)

        favorites = status.favourites_count
        fav_ct_list.append(favorites)

        handleslist.append(handle)

    except:
        pass


@HomeDepotCareer
@macysJOBS
@lowescareers
@MicrosoftJobs
@CVSCareers
@cvshealth
@walgreens


From cffi callback <function _verify_callback at 0x000000000D23B400>:
Traceback (most recent call last):
  File "C:\Users\boyerm\AppData\Local\Continuum\Anaconda3\lib\site-packages\OpenSSL\SSL.py", line 200, in wrapper
    @wraps(callback)
KeyboardInterrupt


In [262]:
print(handleslist)

[]


In [246]:
twitter_info = pd.DataFrame()
twitter_info.shape
len(handleslist)

0

In [248]:
twitter_info = pd.DataFrame({'handle': handleslist,'friends':friendsct,'followers':followerscount, 'verified': verified_list,
                            'year_started':year_started_list,'status_count':status_ct_list, 'fav_count':fav_ct_list})  

twitter_info['handle']=twitter_info['handle'].str.lower()
twitter_info.shape

(363, 7)

In [250]:
twitter_info = twitter_info[twitter_info['friends'] > 0]

twitter_info['influence_ratio'] =twitter_info['followers']/ twitter_info['friends']


In [251]:
print(max(twitter_info['influence_ratio']))
print(min(twitter_info['influence_ratio']))


71492.5384615
0.24512195122


In [252]:
watson_glassdoor_tw = pd.merge(watson_glassdoor,twitter_info, how = 'outer', on='handle')
watson_glassdoor_tw = watson_glassdoor_tw.drop_duplicates(keep=False)

In [253]:
filename = 'watson_glassdoor_tw.csv'
watson_glassdoor_tw.to_csv(filename, index=False, encoding='utf-8')